In [24]:
from pypdf import PdfReader

In [25]:
# For LangChain 0.0.348
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [26]:
file_path = '/content/drive/MyDrive/broadway py w ai/Human Body Chatbot Training.pdf'

In [27]:
from pypdf import PdfReader

def pdf_reader_safe(file_path):
    reader = PdfReader(file_path)
    text = ""

    for i, page in enumerate(reader.pages):
      page_text = page.extract_text()
      if page_text:
        text += page_text + "\n"



    return text


In [28]:
text = pdf_reader_safe(file_path)

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [30]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
chunk = splitter.split_text(text)

In [31]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [32]:
from langchain_core.documents import Document

documents = [Document(page_content=t) for t in chunk]
vectorstore = FAISS.from_documents(documents, embeddings)

In [33]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [34]:
qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256
)

llm = HuggingFacePipeline(pipeline=qa_pipeline)

Device set to use cuda:0


In [35]:
query = "What is the document about?"
documents = retriever.invoke(query)

context = "\n".join([d.page_content for d in documents])
answer = llm.invoke(f"Answer based on context:\n{context}\n\nQuestion:{query}")

print(answer)

human body


In [48]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    get_chat_history=lambda h: h,

)

# Conversation
print(conversation_chain.invoke({"question": "What is this document about?"})["answer"])
print(conversation_chain.invoke({"question": "Tell me about human bones."})["answer"])
print(conversation_chain.invoke({"question": "How many bones are there?"})["answer"])
print(conversation_chain.invoke({"question": "In my last question, what topic was I asking about?"})["answer"])

Complete Human Body Knowledge Base for Chatbot Training
The skeletal system provides structure, support, and protection. It consists of 206 bones in an adult human. Bones store minerals such as calcium and phosphorus. The skeletal system produces blood cells in the bone marrow. Joints connect bones and allow movement. Bones store minerals such as calcium and phosphorus. The skeletal system produces blood cells in the bone marrow. Joints connect bones and allow movement. Bones store minerals such as calcium and phosphorus. The skeletal system produces blood cells in the bone marrow. Joints connect bones and allow movement. Bones store minerals such as calcium and phosphorus. The skeletal system produces blood cells in the bone marrow. Joints connect bones and allow movement. Bones store minerals such as calcium and phosphorus. The skeletal system produces blood cells in the bone marrow. Joints connect bones and allow movement. Bones store minerals such as calcium and phosphorus. The ske